In [1]:
# TRANSFER LEARNING FINE TUNING COLORACTEL CANCER PATCH CLASSIFICATION 
import numpy as np
import math
from sklearn.model_selection import train_test_split, StratifiedKFold
import tensorflow.keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, Lambda, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, AveragePooling2D, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
#from keras.utils import np_utils
%matplotlib inline
tensorflow.keras.backend.set_image_data_format('channels_last')
import random
from tensorflow.keras.layers import LeakyReLU

In [2]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50

base_model=ResNet50(weights='imagenet',include_top=False, input_shape=(224,224,3))

for layer in base_model.layers:
    layer.trainable=False

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024)(x)
x=LeakyReLU(alpha=0.3)(x)
x=Dropout(0.5)(x)
x=Dense(512)(x) 
x=LeakyReLU(alpha=0.3)(x)
preds=Dense(8,activation='softmax')(x)

model=Model(inputs=base_model.input,outputs=preds)

In [3]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [4]:
len(model.layers)

182

In [5]:
# Data augmentation
batch_size=4

train_datagen = ImageDataGenerator(rescale=1./255,
                         horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.2,
                         height_shift_range = 0.2,
                         zoom_range = 0.2,
                         rotation_range = 180,
                         shear_range=0.2
                        )

valid_datagen = ImageDataGenerator(#featurewise_center=True,
                                   rescale=1./255,
                                  )

train_generator = train_datagen.flow_from_directory('data01/train', 
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True)
valid_generator = valid_datagen.flow_from_directory('data01/valid', 
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True)


Found 3496 images belonging to 8 classes.
Found 752 images belonging to 8 classes.


In [6]:
#Callbacks
def get_callbacks(name_weights, patience_lr):
    mcp_save = ModelCheckpoint(name_weights, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=2, min_delta=1e-4, mode='min')
    early_stop_cr=EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=2, mode='auto', restore_best_weights=True)
    return [mcp_save, reduce_lr_loss, early_stop_cr]


In [7]:
from tensorflow.keras import optimizers
model.compile(optimizer=optimizers.Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy']) 

C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [8]:
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size

name_weights = "resnet50_crc_model_weights_13jan22.h5"
callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=3,
                   verbose=2,
                   callbacks = callbacks)

C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/3
874/874 - 173s - loss: 2.1262 - accuracy: 0.2417 - val_loss: 1.6601 - val_accuracy: 0.3072


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/3
874/874 - 77s - loss: 1.8050 - accuracy: 0.2863 - val_loss: 1.6308 - val_accuracy: 0.2660
Epoch 3/3
874/874 - 77s - loss: 1.7048 - accuracy: 0.2978 - val_loss: 1.5502 - val_accuracy: 0.2606


In [9]:
len(model.layers)

182

In [10]:
#Training stage 2

for layer in model.layers[:135]:
    layer.trainable=False
for layer in model.layers[135:]:  #last ~25% layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=10,
                   verbose=2,
                   callbacks = callbacks)

Epoch 1/10
874/874 - 84s - loss: 2.0607 - accuracy: 0.2929 - val_loss: 1.5407 - val_accuracy: 0.4096
Epoch 2/10
874/874 - 75s - loss: 1.5900 - accuracy: 0.3207 - val_loss: 2.9396 - val_accuracy: 0.2832
Epoch 3/10
874/874 - 76s - loss: 1.5155 - accuracy: 0.3558 - val_loss: 11.0314 - val_accuracy: 0.1250
Epoch 4/10
874/874 - 75s - loss: 1.4408 - accuracy: 0.4068 - val_loss: 1.3255 - val_accuracy: 0.4162
Epoch 5/10
874/874 - 73s - loss: 1.4388 - accuracy: 0.4113 - val_loss: 1.8780 - val_accuracy: 0.2965
Epoch 6/10
874/874 - 74s - loss: 1.3621 - accuracy: 0.4365 - val_loss: 1.2912 - val_accuracy: 0.4269
Epoch 7/10
874/874 - 74s - loss: 1.3112 - accuracy: 0.4700 - val_loss: 8.7182 - val_accuracy: 0.1489
Epoch 8/10
874/874 - 73s - loss: 1.2745 - accuracy: 0.4843 - val_loss: 1.7479 - val_accuracy: 0.3777
Epoch 9/10
874/874 - 74s - loss: 1.2463 - accuracy: 0.4969 - val_loss: 1.2020 - val_accuracy: 0.4535
Epoch 10/10
874/874 - 75s - loss: 1.1896 - accuracy: 0.5154 - val_loss: 5.6377 - val_accur

In [11]:
#Training stage 3
for layer in model.layers:
    layer.trainable=True

model.compile(optimizer=optimizers.Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=150,
                   verbose=2,
                   callbacks = callbacks)

Epoch 1/150
874/874 - 108s - loss: 1.7854 - accuracy: 0.4399 - val_loss: 42.4027 - val_accuracy: 0.2553
Epoch 2/150
874/874 - 96s - loss: 1.0520 - accuracy: 0.6061 - val_loss: 3.8472 - val_accuracy: 0.4880
Epoch 3/150
874/874 - 96s - loss: 0.8793 - accuracy: 0.6670 - val_loss: 3.5683 - val_accuracy: 0.5638
Epoch 4/150
874/874 - 92s - loss: 0.7793 - accuracy: 0.7217 - val_loss: 1.7514 - val_accuracy: 0.7221
Epoch 5/150
874/874 - 92s - loss: 0.6956 - accuracy: 0.7431 - val_loss: 2.6158 - val_accuracy: 0.7048
Epoch 6/150
874/874 - 88s - loss: 0.6383 - accuracy: 0.7646 - val_loss: 1.0449 - val_accuracy: 0.8098
Epoch 7/150
874/874 - 95s - loss: 0.5691 - accuracy: 0.7886 - val_loss: 2.4957 - val_accuracy: 0.8072
Epoch 8/150
874/874 - 94s - loss: 0.5467 - accuracy: 0.8064 - val_loss: 1.4320 - val_accuracy: 0.7686
Epoch 9/150
874/874 - 97s - loss: 0.5082 - accuracy: 0.8149 - val_loss: 1.4137 - val_accuracy: 0.7247
Epoch 10/150
874/874 - 96s - loss: 0.4892 - accuracy: 0.8192 - val_loss: 0.5714 

In [14]:
# Validation accuracy, validation data confusion matrix, area under the roc score
valid_generator = valid_datagen.flow_from_directory('data01/valid', 
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=False)

# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score

#https://github.com/bhattbhavesh91/imbalance_class_sklearn/blob/master/imbalance_class_undersampling_oversampling.ipynb

Y_pred = model.predict_generator(valid_generator)
y_pred=np.argmax(Y_pred,axis=1)

ras=roc_auc_score(valid_generator.classes, Y_pred,multi_class='ovr')
print('Confusion Matrix:')
print(confusion_matrix(valid_generator.classes, y_pred))
print('Accuracy:', accuracy_score(valid_generator.classes, y_pred))
print('One versus rest AUC score:', ras)

Found 752 images belonging to 8 classes.
Confusion Matrix:
[[85  1  2  1  0  4  0  1]
 [ 0 86  4  0  3  0  1  0]
 [ 3 20 69  2  0  0  0  0]
 [ 0  0  4 86  0  4  0  0]
 [ 0 15  2  0 71  4  0  2]
 [ 2  0  0  0  0 92  0  0]
 [ 0  0  0  0  2  0 89  3]
 [ 0  8  0  0  0  0  2 84]]
Accuracy: 0.8803191489361702
One versus rest AUC score: 0.9835393358339262


In [16]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory('data01/test', 
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=False)

TEST_pred = model.predict_generator(test_generator)
test_pred=np.argmax(TEST_pred,axis=1)

print('Confusion Matrix:')
ras=roc_auc_score(test_generator.classes, TEST_pred,multi_class='ovr')
print(confusion_matrix(test_generator.classes, test_pred))
print('Accuracy:') 
print(accuracy_score(test_generator.classes, test_pred))
print('One versus rest AUC score:', ras)


Found 752 images belonging to 8 classes.


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


Confusion Matrix:
[[81  1  5  1  0  6  0  0]
 [ 0 87  3  0  2  0  2  0]
 [ 2 18 64  6  2  2  0  0]
 [ 0  0 10 79  0  5  0  0]
 [ 0 23  2  0 65  2  2  0]
 [ 0  0  1  0  0 93  0  0]
 [ 0  0  0  0  0  0 93  1]
 [ 0  6  0  0  0  0  0 88]]
Accuracy:
0.8643617021276596
One versus rest AUC score: 0.9834372776951432
